# Processing protein input data, retreiving sequences 

In [1]:
from Bio import Entrez
from Bio import SeqIO
from Bio.Seq import Seq
import pandas as pd

# returns fasta sequence from accession code
def fetch_sequence(accession_code):
    Entrez.email = "ug19932@bristol.ac.uk"
    handle = Entrez.efetch(db="protein", id=accession_code, rettype="fasta", retmode="text")
    record = SeqIO.read(handle, "fasta")
    sequence = record.seq
    return sequence

# reading in list of accession codes
protein_df = pd.read_csv('protein_codes.csv').dropna(how='all') 

# adding fasta sequence to protein_df
temp_seq_list = []
for i in range(protein_df['access_code'].count()):
    if protein_df.loc[i,'access_code'].count('#') == 0:
        temp_seq_list.append(fetch_sequence(protein_df.loc[i,'access_code']))
    else:
        bio_sequence = Seq(protein_df.loc[i,'sequence'])       
        temp_seq_list.append(bio_sequence)      

protein_df['sequence'] = temp_seq_list


Adding any other sequences 

In [2]:
print(type(protein_df.loc[0,'sequence']))

<class 'Bio.Seq.Seq'>


# Processing histogram data 

In [3]:
motif_file = open('motif.txt', 'r')
motif_list = motif_file.read().split('\n')
motif_file.close()
motif_list

['YFASGGTREGL',
 'YFASGGTQEGL',
 'YFCSGGTKEGL',
 'YFCTGGTQDML',
 'YFASGGTQEAL',
 'YFSTGGDQASL',
 'YFASGGTKESL',
 'FFCSGGTRDAL',
 'YFALGGTKAGL',
 'YFCCGGTKEEL',
 'YFASGGSQEAL',
 'FFCSGGTREGL',
 'IFLAGGTMEEL',
 'TKCCGGNKEEL',
 'LFCAGGSQREL']

# Making dictionary with motif data

In [4]:
motif_pos_list = []
for i_position in range(len(motif_list[0])):
    temp_string = []
    for i_motifs in range(len(motif_list)):
        temp_string.append(motif_list[i_motifs][i_position])
    motif_pos_list.append(''.join(temp_string))
print(motif_pos_list)    
motif_df = pd.DataFrame(columns=['position','A','R','N','D','C','Q','G','E','H','I','L','K','M','F','P','S','T','W','Y','V'])
all_amino_acids = ['A','R','N','D','C','Q','G','E','H','I','L','K','M','F','P','S','T','W','Y','V']

for i_position in range(len(motif_pos_list)):
    new_row = {'position':i_position,
               'A':motif_pos_list[i_position].count('A'),
               'R':motif_pos_list[i_position].count('R'),
               'N':motif_pos_list[i_position].count('N'),
               'D':motif_pos_list[i_position].count('D'),
               'C':motif_pos_list[i_position].count('C'),
               'Q':motif_pos_list[i_position].count('Q'),
               'G':motif_pos_list[i_position].count('G'),
               'E':motif_pos_list[i_position].count('E'),
               'H':motif_pos_list[i_position].count('H'),
               'I':motif_pos_list[i_position].count('I'),
               'L':motif_pos_list[i_position].count('L'),
               'K':motif_pos_list[i_position].count('K'),
               'M':motif_pos_list[i_position].count('M'),
               'F':motif_pos_list[i_position].count('F'),
               'P':motif_pos_list[i_position].count('P'),
               'S':motif_pos_list[i_position].count('S'),
               'T':motif_pos_list[i_position].count('T'),
               'W':motif_pos_list[i_position].count('W'),
               'Y':motif_pos_list[i_position].count('Y'),
               'V':motif_pos_list[i_position].count('V')}
    motif_df.loc[len(motif_df)] = new_row
    
motif_df.set_index('position',inplace=True, drop=True)    
max_values = motif_df.max(axis=1)
motif_df = motif_df.divide(max_values, axis="rows")
motif_df_full = motif_df

['YYYYYYYFYYYFITL', 'FFFFFFFFFFFFFKF', 'AACCASACACACLCC', 'SSSTSTSSLCSSACA', 'GGGGGGGGGGGGGGG', 'GGGGGGGGGGGGGGG', 'TTTTTDTTTTSTTNS', 'RQKQQQKRKKQRMKQ', 'EEEDEAEDAEEEEER', 'GGGMASSAGEAGEEE', 'LLLLLLLLLLLLLLL']


In [117]:
motif_dict = {}

for i_position in range(len(motif_df)):
    
    for i_aminoacids in range(len(motif_df.columns)):
        aminoacid = motif_df.columns[i_aminoacids]
        percentage = motif_df.loc[i_position,aminoacid]
        
        if motif_df.loc[i_position,aminoacid] != 0:
            if i_position in motif_dict:
                motif_dict[i_position].update({aminoacid:percentage})
            else: 
                motif_dict[i_position] = {aminoacid:percentage}
                
motif_dict = {k: dict(sorted(v.items(), key=lambda x: x[1], reverse=True)) for k, v in motif_dict.items()}           

motif_dict


{0: {'Y': 1.0, 'F': 0.2, 'I': 0.1, 'L': 0.1, 'T': 0.1},
 1: {'F': 1.0, 'K': 0.07142857142857142},
 2: {'C': 1.0,
  'A': 0.8571428571428571,
  'L': 0.14285714285714285,
  'S': 0.14285714285714285},
 3: {'S': 1.0, 'A': 0.25, 'C': 0.25, 'T': 0.25, 'L': 0.125},
 4: {'G': 1.0},
 5: {'G': 1.0},
 6: {'T': 1.0,
  'S': 0.18181818181818182,
  'N': 0.09090909090909091,
  'D': 0.09090909090909091},
 7: {'Q': 1.0, 'K': 0.8333333333333334, 'R': 0.5, 'M': 0.16666666666666666},
 8: {'E': 1.0, 'A': 0.2, 'D': 0.2, 'R': 0.1},
 9: {'G': 1.0, 'E': 0.8, 'A': 0.6, 'S': 0.4, 'M': 0.2},
 10: {'L': 1.0}}

# Searching through

In [21]:
#looping through each protein in the df
results_df = pd.DataFrame(columns=['accession code', 'found position', 'found motif','score'])

for i_protein in range(protein_df['access_code'].count()):
    sequence = protein_df.loc[i_protein,'sequence']
    motif_df = motif_df_full
    #defining dataframe 
    df = pd.DataFrame(columns=['length','location','score','rel score', 'seq'])
     
    for i_remove in range(len(motif_df)):
        motif_dict = {}

        for i_position in range(len(motif_df)):

            for i_aminoacids in range(len(motif_df.columns)):
                aminoacid = motif_df.columns[i_aminoacids]
                percentage = motif_df.loc[i_position,aminoacid]

                if motif_df.loc[i_position,aminoacid] != 0:
                    if i_position in motif_dict:
                        motif_dict[i_position].update({aminoacid:percentage})
                    else: 
                        motif_dict[i_position] = {aminoacid:percentage}
                
        motif_dict = {k: dict(sorted(v.items(), key=lambda x: x[1], reverse=True)) for k, v in motif_dict.items()}           
        
    
        #Makes list of possible amino acids in each position 
        aa_2d_list = [] # list of amino acids 
        for i_motif_pos in range(len(motif_dict)):
            aa_2d_list.append([key[0] for key in motif_dict[i_motif_pos].keys()])

        motif_length = len(aa_2d_list)

        #Searching for the first aa in motif and making a list with locations 
        first_aa_location = []   
        for i_aa in range(len(aa_2d_list[0])):
            seq_pointer = 0 
            location = 0 
            aa = aa_2d_list[0][i_aa] 
            temp_first_aa_location = [] #to hold 1D list 

            while location != -1:
                location = sequence.find(aa, seq_pointer)
                seq_pointer = location + 1
                if location != -1:
                    temp_first_aa_location.append(location)
            first_aa_location.append(temp_first_aa_location)
        print(first_aa_location)

        

        #original score set 
        for i_first_aa in range(len(first_aa_location)): #for aa in first position of motif
            aa = aa_2d_list[0][i_first_aa]
            #print(aa)



            #getting location of first aa 
            for i_location in range(len(first_aa_location[i_first_aa])): #for each location (in each first position loop set above)
                score = motif_dict[0][aa]
                length_count = 1
                seq_pointer = first_aa_location[i_first_aa][i_location] #set the seq pointer to location1
                found_seq = sequence[seq_pointer]
                #print(seq_pointer)
                isfound = True 
                for motif_pos in range(motif_length): #for the length of the motif 
                    if motif_pos+1 == motif_length: #if reached the end of the motif
                        score = score / motif_length
                        rel_score = (score*motif_length)/length_count
                        if length_count > 2:
                            
                            new_row = {'length': length_count,'location': first_aa_location[i_first_aa][i_location],  'score': score, 'rel score': rel_score, 'seq': found_seq}
                            df.loc[len(df)] = new_row
                    elif seq_pointer+1 == len(sequence):
                        isfound = False
                    elif isfound == True:
                        seq_pointer += 1
                        isfound = False
                        for next_pos in range(len(aa_2d_list[motif_pos+1])): #for the length of the next position in the motif 
                            if isfound == False: 
                                if  sequence[seq_pointer] == aa_2d_list[motif_pos+1][next_pos]:                         
                                    score = score + motif_dict[motif_pos+1][aa_2d_list[motif_pos+1][next_pos]]
                                    length_count += 1
                                    isfound = True
                                    found_seq = ''.join([found_seq, aa_2d_list[motif_pos+1][next_pos]])

                    else:
                        score = score  
        motif_df = motif_df.drop(index=0) 
        motif_df = motif_df.reset_index(drop=True)
        
        
    ##makes textfile with incomplete found sequences  
    df = df.sort_values(by='length', ascending=False)
    df_string = df.to_string(index = False)
    f = open("foundsequences.txt", "a")
    f.write(df_string)
    f.write('\n')
    f.write('\n')
    f.close()
    
    found_pos = []
    found_score = []
    found_motif = []
    for i_found in range(len(df)):
        if df.loc[i_found,'length'] == len(motif_df_full):
            found_pos.append(df.loc[i_found, 'location'])
            found_motif.append(df.loc[i_found, 'seq'])
            found_score.append(df.loc[i_found, 'score'])           
            print(found_pos)

    new_row = {'accession code': protein_df['access_code'][i_protein], 
               'found position': found_pos, 
               'found motif': found_motif,
               'score': found_score}
    results_df.loc[len(results_df)] = new_row
    
results_df_string = results_df.to_string(index = False)
f = open("results.txt", "w")
f.write(results_df_string)
f.close()





            
   
        
        

[[29, 42, 44, 55, 101, 137, 140, 165, 216, 246, 247, 303, 331, 332, 364, 373, 378, 531, 557, 586, 637, 640, 662, 687, 732, 864, 900, 916, 1008, 1021, 1137, 1149, 1202, 1212, 1251, 1386, 1397, 1408, 1520, 1552, 1591, 1606, 1628, 1698, 1708, 1711, 1722, 1745, 1766, 1878, 1890, 1899, 2004, 2053, 2063, 2084, 2098, 2114, 2190, 2215, 2248, 2260, 2287, 2347, 2356, 2398, 2455, 2529, 2567, 2640, 2669, 2794, 2796, 2805, 2923, 2925, 2934, 3032, 3040, 3043, 3109, 3113, 3159, 3205, 3317, 3328, 3397, 3400, 3422, 3512, 3556, 3701, 3736, 3746, 3776, 3823, 3842, 3848, 3861, 3880, 3986, 4019, 4029, 4031, 4160, 4171, 4232, 4240, 4242, 4268, 4402, 4440, 4446, 4486, 4501, 4538, 4594, 4605, 4686, 4699, 4706, 4841, 4851, 4906, 4909, 4940, 5000, 5005, 5079, 5081, 5121], [34, 37, 114, 150, 183, 215, 219, 251, 252, 265, 275, 309, 362, 439, 448, 472, 489, 496, 500, 501, 518, 549, 573, 587, 590, 655, 704, 755, 764, 813, 818, 850, 946, 956, 970, 1023, 1064, 1073, 1074, 1146, 1205, 1233, 1268, 1288, 1328, 1353, 136

In [20]:

print(df.style.hide(axis="index"))